In [1]:
from functions import get_data
import pandas as pd
import numpy as np


In [2]:
us_data=get_data('GOLD')

In [13]:
def x_points(data,index):
    # index=index.tolist()[0]
    val_h=data.iloc[index]
    start_date = val_h['mtime'].date()
    t=val_h['mtime'].time()
    filtered_data = data[(data['mtime'].dt.date == start_date) & (data.index <= index)]
    # print(filtered_data)
    # print(f'filtered_data:\n{filtered_data.head()}')
    max_value = filtered_data['Close'].max()
    min_value = filtered_data['Close'].min()
    return max_value, min_value
    
    

In [14]:
def x_points2(data,index):
    # index=index.tolist()[0]
    val_h=data.iloc[index]
    start_date = val_h['mtime'].date()
    t=val_h['mtime'].time()
    filtered_data = data[(data['mtime'].dt.date == start_date) & (data.index <= index)]
    # print(filtered_data)
    # print(f'filtered_data:\n{filtered_data.head()}')
    max_value = filtered_data['Close'].max()
    min_value = filtered_data['Close'].min()
    return max_value, min_value
    
    

In [3]:
us_data['mtime']=pd.to_datetime(us_data['time'],unit='s')


In [12]:
def distance_checker(price,value,lim):
    plh=price-value
    plh=abs(plh)
    if plh >lim:
        return 1
    else:
        return 0
    
    

In [11]:
def find_status(data,index,window,size):
    cspreads=[]
    # print(index,window,size)
    # print("fstatus here1")
    for i in range(window):
        datapoint=data.iloc[index-i]
        close=datapoint['Close']
        openv=datapoint['Open']
        cspread=abs(close-openv)
        cspreads.append(cspread)
    # print("fstatus here2")
    if max(cspreads)>size:
        return 1
    else:
        return 0
    

In [10]:
from functions import calc_prof

In [15]:
def daily(data,starttime,adjtime,lim,window,c_size,sl_adj,multiplier):
    trades=[]
    trade=0
    opendate=None
    openprice=0
    tp=0
    sl=0
    for i in range(len(data)):
        val_h=data.loc[i]
        time=val_h['mtime'].time()
        adj_time = pd.to_datetime(adjtime).time()# adtime being such astring '03:00:00'
        if time<adj_time:
            continue
        else:
            
            tday=val_h['mtime'].date()
            target_datetime = pd.to_datetime(f'{tday} {starttime}')
            # print(target_datetime)
            val=data[data['mtime'] == target_datetime]
            # print(val)
            # print(val.index)
            
            startindex=val.index
            ind=startindex.tolist()[0]
            begin_max_value,begin_min_value=x_points(data,ind)
            maxv=begin_max_value
            # print(maxv)
            minv=begin_min_value
            # print(minv)
            
            # print(f'tade{trade}')
            price=val_h['Close']
            if trade==0:
                if price>maxv:
                    if distance_checker(price,maxv,lim)==0:
                        maxv=price
                        continue
                    elif distance_checker(price,maxv,lim)==1:
                        # print('i am here 4')
                        if find_status(data,i,window,c_size)==1:
                            # print('HEREEE')
                            # print(ind)
                            trade=-1
                            openprice=price
                            opendate=val_h['time']
                            lowp=val_h['Low']
                            # print('i am here ')
                            sl=lowp-sl_adj
                            expectedloss=price-sl
                            target_tp=expectedloss*multiplier
                            tp=price+target_tp
                            # print(trade)

                            
                elif price<minv:
                    if distance_checker(price,minv,lim)==0:
                        minv=price
                        continue
                    elif distance_checker(price,minv,lim)==1:
                        # print('i am here3 ')
                        if find_status(data,i,window,c_size)==1:
                            # print('found me 1')
                            trade=1
                            openprice=price
                            # print(trade)
                            opendate=val_h['time']
                            # print('i am here2 ')
                            highp=val_h['High']
                            sl=highp-sl_adj
                            expectedloss=sl-price
                            target_tp=expectedloss*multiplier
                            tp=price-target_tp
            elif trade==1:
                # print('i am here8 ')
                if price>tp:
                    # print('i am here 6')
                    closedate=val_h['time'] 
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
                elif price<sl:
                    # print('i am here7 ')
                    closedate=val_h['time']
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
            elif trade==-1:
                # print('i am here 9 ')
                if price<tp:
                    # print('i am here 10 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
                elif price>sl:
                    # print('i am here 11 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
    return trades
                    
                

In [23]:
def daily2(data,starttime,adjtime,lim,window,c_size,sl_adj,multiplier):
    trades=[]
    trade=0
    opendate=None
    openprice=0
    tp=0
    sl=0
    for i in range(len(data)):
        val_h=data.loc[i]
        time=val_h['mtime'].time()
        adj_time = pd.to_datetime(adjtime).time()# adtime being such astring '03:00:00'
        if time<adj_time:
            continue
        else:
            
            tday=val_h['mtime'].date()
            target_datetime = pd.to_datetime(f'{tday} {starttime}')
            # print(target_datetime)
            val=data[data['mtime'] == target_datetime]
            # print(val)
            # print(val.index)
            
            startindex=val.index
            ind=startindex.tolist()[0]
            begin_max_value,begin_min_value=x_points2(data,ind)
            maxv=begin_max_value
            # print(maxv)
            minv=begin_min_value
            # print(minv)
            
            # print(f'tade{trade}')
            price=val_h['Close']
            if trade==0:
                if price>maxv:
                    if distance_checker(price,maxv,lim)==0:
                        maxv=price
                        continue
                    elif distance_checker(price,maxv,lim)==1:
                        # print('i am here 4')
                        if find_status(data,i,window,c_size)==1:
                            # print('HEREEE')
                            # print(ind)
                            trade=-1
                            openprice=price
                            opendate=val_h['time']
                            lowp=val_h['Low']
                            # print('i am here ')
                            sl=lowp-sl_adj
                            expectedloss=price-sl
                            target_tp=expectedloss*multiplier
                            tp=price+target_tp
                            # print(trade)

                            
                elif price<minv:
                    if distance_checker(price,minv,lim)==0:
                        minv=price
                        continue
                    elif distance_checker(price,minv,lim)==1:
                        # print('i am here3 ')
                        if find_status(data,i,window,c_size)==1:
                            # print('found me 1')
                            trade=1
                            openprice=price
                            # print(trade)
                            opendate=val_h['time']
                            # print('i am here2 ')
                            highp=val_h['High']
                            sl=highp-sl_adj
                            expectedloss=sl-price
                            target_tp=expectedloss*multiplier
                            tp=price-target_tp
            elif trade==1:
                # print('i am here8 ')
                if price>tp:
                    # print('i am here 6')
                    closedate=val_h['time'] 
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
                elif price<sl:
                    # print('i am here7 ')
                    closedate=val_h['time']
                    profit=calc_prof(price,openprice)
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
            elif trade==-1:
                # print('i am here 9 ')
                if price<tp:
                    # print('i am here 10 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
                elif price>sl:
                    # print('i am here 11 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
    return trades
                    
                

In [9]:
from enum import Enum

class TradeStatus(Enum):
    BUY = 1
    SELL = -1
    NONE = 0

def daily3(data,  adjtime, lim, window, c_size, sl_adj, multiplier):
    trades = []
    trade = TradeStatus.NONE
    opendate = None
    openprice = 0
    tp = 0
    sl = 0

    for i, val_h in data.iterrows():
        time = val_h['mtime'].time()
        adj_time = pd.to_datetime(adjtime).time()

        if time < adj_time:
            continue

        tday = val_h['mtime'].date()
        target_datetime = pd.to_datetime(f'{tday} {adjtime}')
        val = data[data['mtime'] == target_datetime]
        
        startindex = val.index
        ind = startindex.tolist()[0]
        begin_max_value, begin_min_value = x_points2(data, ind)
        maxv, minv = begin_max_value, begin_min_value

        price = val_h['Close']
        dist_check = distance_checker(price, maxv if trade == TradeStatus.SELL else minv, lim)

        if trade == TradeStatus.NONE:
            if dist_check == 1 and find_status(data, i, window, c_size) == 1:
                if price > maxv:
                    trade = TradeStatus.SELL
                elif price < minv:
                    trade = TradeStatus.BUY

                openprice = price
                opendate = val_h['time']
                sl = (val_h['High'] if trade == TradeStatus.BUY else val_h['Low']) - sl_adj
                expected_loss = abs(sl - price)
                target_tp = expected_loss * multiplier
                tp = price + target_tp if trade == TradeStatus.SELL else price - target_tp

        elif trade in [TradeStatus.BUY, TradeStatus.SELL]:
            if (price > tp and trade == TradeStatus.BUY) or (price < sl and trade == TradeStatus.SELL):
                closedate = val_h['time']
                profit = calc_prof(price, openprice) if trade == TradeStatus.BUY else calc_prof(openprice, price)
                trades.append(('Buy' if trade == TradeStatus.BUY else 'Sell', opendate, closedate, profit))
                trade = TradeStatus.NONE

    return trades


In [4]:
us_data2 = us_data[us_data['mtime'].dt.date != pd.to_datetime('2015-01-02').date()]


In [5]:

testing_dt=us_data2[:10000]



In [16]:
testing_dt.reset_index(drop=True, inplace=True)

In [17]:
import cProfile

In [24]:
cProfile.run('daily2(testing_dt,"03:00:00","03:00:00",2.0,4,4.5,2.5,2.5)')

         40739985 function calls (40158335 primitive calls) in 91.922 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4073    0.170    0.000    4.544    0.001 140914320.py:1(find_status)
    10743    0.016    0.000    0.021    0.000 1705051663.py:1(distance_checker)
        1    0.756    0.756   91.921   91.921 2384412996.py:1(daily2)
     8996    1.117    0.000   52.005    0.006 3078399005.py:1(x_points2)
     8996    0.037    0.000    0.106    0.000 <__array_function__ internals>:177(copyto)
    35288    0.094    0.000    0.792    0.000 <__array_function__ internals>:177(result_type)
     8996    0.019    0.000    0.025    0.000 <frozen importlib._bootstrap>:1053(_handle_fromlist)
        1    0.000    0.000   91.921   91.921 <string>:1(<module>)
   107952    0.107    0.000    0.107    0.000 __init__.py:33(using_copy_on_write)
    44980    0.031    0.000    0.223    0.000 _methods.py:39(_amax)
    44980    0.031    0

In [25]:
cProfile.run('daily(testing_dt,"03:00:00","03:00:00",2.0,4,4.5,2.5,2.5)')

         40739985 function calls (40158335 primitive calls) in 86.554 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4073    0.159    0.000    4.171    0.001 140914320.py:1(find_status)
     8996    1.041    0.000   49.510    0.006 1664280811.py:1(x_points)
    10743    0.017    0.000    0.021    0.000 1705051663.py:1(distance_checker)
        1    0.677    0.677   86.554   86.554 2227302182.py:1(daily)
     8996    0.033    0.000    0.096    0.000 <__array_function__ internals>:177(copyto)
    35288    0.085    0.000    0.742    0.000 <__array_function__ internals>:177(result_type)
     8996    0.019    0.000    0.024    0.000 <frozen importlib._bootstrap>:1053(_handle_fromlist)
        1    0.000    0.000   86.554   86.554 <string>:1(<module>)
   107952    0.097    0.000    0.097    0.000 __init__.py:33(using_copy_on_write)
    44980    0.029    0.000    0.212    0.000 _methods.py:39(_amax)
    44980    0.031    0.0

In [21]:
cProfile.run('daily3(testing_dt,"03:00:00",2.0,4,4.5,2.5,2.5)')

         37381122 function calls (36823642 primitive calls) in 124.951 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      392    0.024    0.000    0.654    0.002 140914320.py:1(find_status)
     8996    0.034    0.000    0.042    0.000 1705051663.py:1(distance_checker)
     8996    1.451    0.000   74.334    0.008 3078399005.py:1(x_points2)
        1    1.037    1.037  124.949  124.949 537257439.py:8(daily3)
    18996    0.092    0.000    0.253    0.000 <__array_function__ internals>:177(copyto)
    10565    0.044    0.000    0.391    0.000 <__array_function__ internals>:177(result_type)
     8996    0.027    0.000    0.034    0.000 <frozen importlib._bootstrap>:1053(_handle_fromlist)
        1    0.002    0.002  124.951  124.951 <string>:1(<module>)
   117953    0.165    0.000    0.165    0.000 __init__.py:33(using_copy_on_write)
    44980    0.048    0.000    0.347    0.000 _methods.py:39(_amax)
    44980    0.047    0

In [ ]:
y=daily3(testing_dt,'03:00:00',3.0,4,4.5,2.5,2.5)

In [ ]:
times=['03:00:00','04:00:00','04:00:00','05:00:00','06:00:00','07:00:00','08:00:00']
distancelim=[2.5,3.0,4.0,5.5,6.5,7.5,9.0,8.0]
candlewindowlimit=[2,1,3,4,5,6,7,8,9,10]
candlesizes=[2.5,3.0,4.0,5.5,6.5,7.5,9.0,8.0]
sl_adj=[2.5,3.0,4.0,5.5,6.5,7.5,9.0,8.0]
multiplier=[2.5,3.0,4.0,5.5,6.5,7.5,9.0,8.0]




In [ ]:
times=['03:00:00']
distancelim=[2.5]
candlewindowlimit=[2,1]
candlesizes=[2.5]
sl_adj=[2.5]
multiplier=[2.5]




In [ ]:
import itertools

In [ ]:
combinations = list(itertools.product(times, distancelim, candlewindowlimit, candlesizes, sl_adj, multiplier))

In [ ]:
from functions import evaluate

In [ ]:
import csv

In [ ]:

def daily_move(data,combinations,output_csv):
    result_list=[]
    for values in combinations:
        adjtime, lim, window, c_size, sl_adj, multiplier = values
        name=f'starttime:{adjtime,} window: {window}candlesize{c_size}sl_adj{sl_adj}multiplier{multiplier}'
        result = daily3(data, adjtime, lim, window, c_size, sl_adj, multiplier)
        evaluated_info=evaluate(result,name)
        result_dict1 = {
        'Strategy': name,
        'Take_profit_level':"level1(tp1)",
        'Max_Profit': evaluated_info[0],
        'Num_Buy_Trades': evaluated_info[1],
        'Total_Buy_Profit': evaluated_info[2],
        'Num_Sell_Trades': evaluated_info[3],
        'Total_Sell_Profit': evaluated_info[4],
        'Max_Profit_Index': evaluated_info[5],
        'Index_of_Max_Profit': evaluated_info[6],
        'Max_Duration': evaluated_info[7],
        'ID_Max_Duration': evaluated_info[8],
        'total_profitable_trades':evaluated_info[9],
        'total_unprofitable_trades':evaluated_info[10],
        'total_profitable_trades':evaluated_info[11],
        'total_loss_trades':evaluated_info[12]}

        result_list.append(result_dict1)
        
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = result_list[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for row in result_list:
            writer.writerow(row)
            

In [ ]:
daily_move(testing_dt,combinations,'tesrt.csv')

In [ ]:
from functions import evaluate2

In [ ]:
ty=evaluate2(y,'daily')

In [6]:
import concurrent.futures
from functions import daily_move2,get_data
import pandas as pd
import itertools


In [ ]:

us_data=get_data('GOLD')
us_data['mtime']=pd.to_datetime(us_data['time'],unit='s')
us_data = us_data[us_data['mtime'].dt.date != pd.to_datetime('2015-01-02').date()]
us_data.reset_index(drop=True, inplace=True)

times=['03:00:00']#,'04:00:00','04:00:00','05:00:00','06:00:00','07:00:00','08:00:00']
distancelim=[2.5]#,3.0,4.0,5.5,6.5,7.5,9.0,8.0]
candlewindowlimit=[2]#,1,3,4,5,6,7,8,9,10]
candlesizes=[2.5]#,3.0,4.0,5.5,6.5,7.5,9.0,8.0]
sl_adj=[2.5]#,3.0,4.0,5.5,6.5,7.5,9.0,8.0]
multiplier=[2.5]#,3.0,4.0,5.5,6.5,7.5,9.0,8.0]





combinations = list(itertools.product(times, distancelim, candlewindowlimit, candlesizes, sl_adj, multiplier))
print(len(combinations))

# if __name__ == "__main__":
    
    # Create a ProcessPoolExecutor
result_df=[]
    # with concurrent.futures.ProcessPoolExecutor() as executor:
        # Use map to apply the joined function to each combination in parallel
        # results_list = executor.map(daily_move2, [us_data] * len(combinations), *zip(*combinations))
# a=daily_move2(*len(combinations),*zip(combinations)) 
u=[]  
for combo in combinations:
    # u.append(*combo)
    # print(i)
    a=daily_move2(us_data,*combo)
    result_df.append(a)

# Save the DataFrame to a CSV file
# u



In [7]:
import cProfile

In [8]:
ip=daily_move2(testing_dt,"03:00:00",2.5,2,2.5,2.5,2.5)
ip

d:\needed\dns projects\trade\functions.py:568: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mtime'] = pd.to_datetime(data['mtime'])


KeyboardInterrupt: 

In [26]:
cProfile.run('daily_move2(testing_dt,"03:00:00",2.5,2,2.5,2.5,2.5)')

d:\needed\dns projects\trade\functions.py:568: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mtime'] = pd.to_datetime(data['mtime'])


         26355435 function calls (25890505 primitive calls) in 62.377 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    16306    0.055    0.000    0.147    0.000 <__array_function__ internals>:177(copyto)
     8213    0.025    0.000    0.200    0.000 <__array_function__ internals>:177(result_type)
     8154    0.019    0.000    0.023    0.000 <frozen importlib._bootstrap>:1053(_handle_fromlist)
        1    0.000    0.000   62.377   62.377 <string>:1(<module>)
   105994    0.090    0.000    0.090    0.000 __init__.py:33(using_copy_on_write)
        1    0.000    0.000    0.000    0.000 _exceptions.py:33(find_stack_level)
     9667    0.007    0.000    0.039    0.000 _methods.py:39(_amax)
     9667    0.008    0.000    0.078    0.000 _methods.py:43(_amin)
     1512    0.001    0.000    0.009    0.000 _methods.py:47(_sum)
    91195    0.062    0.000    0.484    0.000 _methods.py:55(_any)
    16306    0.017    0.000    0.241

IndexError: single positional indexer is out-of-bounds